# DES Simulation

This notebook generates the results used in the report for the second assignment of the course 'Stochastic Simulation'.

Authors:

1. Divyaben Hasmukhbhai Gajera
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: divya.gajera@student.uva.nl
- Student Number: 14932644

2. K. López
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: kenia.lopez.sotomayor@student.uva.nl
- Student Number: 12965081

3. T. P. Glansdorp
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: thomas.glansdorp@student.uva.nl
- Student Number: 12748587

In [34]:
import simpy as sp
import numpy as np
import matplotlib.pyplot as plt
import random
import queue

## DES program that implements FIFO scheduling case

In [96]:
class MMnFIFOQueue:
    def __init__(self, env, arrival_rate, service_rate, n):
        self.env = env
        self.server = sp.Resource(env, capacity=n)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.queue = []

        env.process(self.arrive())
        

    def arrive(self):
        customer_count = 0
        while True:
            inter_arrival_time = random.expovariate(self.arrival_rate)
            yield self.env.timeout(inter_arrival_time)
            self.queue.append(self.env.now)
            customer_count += 1
            self.env.process(self.depart(customer_count))
            print(f"Customer {customer_count} arrived at {self.env.now}")

            
    def depart(self, customer_count):
        print(self.queue)
        with self.server.request() as request:
            yield request
            service_time = random.expovariate(self.service_rate)
            yield self.env.timeout(service_time)
            self.queue.pop(0)
            print(f"Customer {customer_count} departed at {self.env.now} with wait_time of {service_time}")


def run_MMn_FIFO_queue(arrival_rate, service_rate, n, simulation_time):
    env = sp.Environment()
    mmn_queue = MMnFIFOQueue(env, arrival_rate, service_rate, n)
    env.run(until=simulation_time)


In [97]:
arrival_rate = 2.0
service_rate = 3.0
simulation_time = 10.0
n = 2

run_MMn_FIFO_queue(arrival_rate, service_rate, n, simulation_time)

Customer 1 arrived at 1.1031092715306303
[1.1031092715306303]
Customer 2 arrived at 1.2700232264634765
[1.1031092715306303, 1.2700232264634765]
Customer 2 departed at 1.2946491980845236 with wait_time of 0.024625971621047155
Customer 3 arrived at 1.3379541421687091
[1.2700232264634765, 1.3379541421687091]
Customer 1 departed at 1.3621056321428375 with wait_time of 0.25899636061220715
Customer 4 arrived at 1.3693797031676418
[1.3379541421687091, 1.3693797031676418]
Customer 4 departed at 1.4111208007265923 with wait_time of 0.041741097558950495
Customer 3 departed at 1.7173676497368666 with wait_time of 0.37941350756815745
Customer 5 arrived at 2.986798564406796
[2.986798564406796]
Customer 5 departed at 3.0565571232605255 with wait_time of 0.06975855885372978
Customer 6 arrived at 3.8823950220388235
[3.8823950220388235]
Customer 7 arrived at 3.9115469834380003
[3.8823950220388235, 3.9115469834380003]
Customer 8 arrived at 3.9128814111940082
[3.8823950220388235, 3.9115469834380003, 3.91

In [1]:
import simpy as sm
import random
import pandas as pd
import csv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
class g:
    arrival_rate = 1
    service_rate = 2
    number_of_servers = 1
    sim_duration = 200
    number_of_runs = 10
    job_length = {}

class Customer:
    def __init__(self, customer_id):
        self.id = customer_id
        self.queue_time = 0

class Queue:
    
    def __init__(self, run_number, queue_type):
        self.env = sm.Environment()
        self.customer_counter = 0
        self.server = sm.Resource(self.env, capacity=g.number_of_servers)
        self.run_number = run_number
        self.mean_queue_time = 0
        self.queue_type = queue_type
        self.results_df = pd.DataFrame()
        self.results_df["Customer ID"] = []
        self.results_df["Start Queueing"] = []
        self.results_df["End Queueing"] = []
        self.results_df["Queueing Time"] = []
        self.results_df["Sojourn Time"] = []
        self.results_df.set_index("Customer ID", inplace=True)
        
    
    # Method that generates customers arrivals
    def generate_arrivals(self):
        while True:
            self.customer_counter += 1
            cp = Customer(self.customer_counter)
            # Run activity generator for this customer
            self.env.process(self.create_queue(cp))

            # sample time until next customer
            t = random.expovariate(g.arrival_rate)

            # Freezr untill that time has passed
            yield self.env.timeout(t)
            
    def create_queue(self, customer):
        start_queue = self.env.now
        # Request a server
        with self.server.request() as req:
            # Freeze until the request can be met 
            yield req
            
            end_queue = self.env.now
            # Sample time spent in server
            if self.queue_type == "MMn":
                sampled_service_time = random.expovariate(g.service_rate)
            elif self.queue_type == "MDn":
                sampled_service_time = 2
            #sampled_service_time = 1
            elif self.queue_type == "MLn":
                rand = random.random()
                if rand < 0.75:
                    sampled_service_time = random.expovariate(1.0)
                else:
                    sampled_service_time = random.expovariate(5.0)

            # Freeze until that time has passed
            yield self.env.timeout(sampled_service_time)
            
            # Calculate time customer was queueing
            out_of_system = self.env.now
            sojourn_time = out_of_system - start_queue
            customer.queue_time = end_queue - start_queue
            df_to_add = pd.DataFrame({"Customer ID":[customer.id], "Start Queueing": [start_queue],
                                     "End Queueing": [end_queue], "Queueing Time": [customer.queue_time],
                                     "Sojourn Time": [sojourn_time]})
            df_to_add.set_index("Customer ID", inplace=True)
            self.results_df = self.results_df.append(df_to_add)
            
    def calculate_mean_queue_time(self):
        self.mean_queue_time = self.results_df["Queueing Time"].mean()
        
    def write_run_results(self):
        with open("trial_results.csv", "a") as f:
            writer = csv.writer(f, delimiter=",")
            results_to_write = [self.run_number, self.mean_queue_time]
            writer.writerow(results_to_write)
            
    def run(self):
        self.env.process(self.generate_arrivals())
        self.env.run(until=g.sim_duration)
        # Calculate run results
        self.calculate_mean_queue_time()
        self.write_run_results()


In [3]:
with open("trial_results.csv", "w") as f:
    writer = csv.writer(f, delimiter=",")
    column_headers = ["Run", "Mean Queue Time"]
    writer.writerow(column_headers)
    
for run in range(g.number_of_runs):
    mu_queue_model = Queue(run, "MDn")
    mu_queue_model.run()

In [4]:
mu_queue_model.results_df

,Start Queueing,End Queueing,Queueing Time,Sojourn Time
Customer ID,,,,
1,0.000000,0.000000,0.000000,2.000000
2,2.548015,2.548015,0.000000,2.000000
3,3.895530,4.548015,0.652485,2.652485
4,4.071226,6.548015,2.476789,4.476789
5,4.321804,8.548015,4.226211,6.226211
...,...,...,...,...
95,86.888302,188.548015,101.659713,103.659713
96,88.657431,190.548015,101.890584,103.890584
97,89.118094,192.548015,103.429922,105.429922


### Statistical significance 

### Numer of measurements required

## DES program that implements shortest job first case for M/M/1

In [5]:
class SJF_queue:
    
    def __init__(self, run_number):
        self.env = sm.Environment()
        self.customer_counter = 0
        self.server = sm.PriorityResource(self.env, capacity=g.number_of_servers)
        self.run_number = run_number
        self.mean_queue_time = 0
        self.results_df = pd.DataFrame()
        self.results_df["Customer ID"] = []
        self.results_df["Start Queueing"] = []
        self.results_df["End Queueing"] = []
        self.results_df["Queueing Time"] = []
        self.results_df.set_index("Customer ID", inplace=True)
        
    
    # Method that generates customers arrivals
    def generate_arrivals(self):
        while True:
            self.customer_counter += 1
            cp = Customer(self.customer_counter)
            # Run activity generator for this customer
            self.env.process(self.create_queue(cp))

            # sample time until next customer
            t = random.expovariate(g.arrival_rate)

            # Freezr untill that time has passed
            yield self.env.timeout(t)
            
    def create_queue(self, customer):
        start_queue = self.env.now
        # Request a server
        sampled_service_time = random.expovariate(g.service_rate)
        with self.server.request(priority = int(sampled_service_time)) as req:
            # Freeze until the request can be met 
            yield req  
            
            end_queue = self.env.now
            # Sample time spent in server
            if self.queue_type == "MMn":
                sampled_service_time = random.expovariate(g.service_rate)
            elif self.queue_type == "MDn":
                sampled_service_time = 2
            #sampled_service_time = 1
            elif self.queue_type == "MLn":
                rand = random.random()
                if rand <= 0.75:
                    sampled_service_time = random.expovariate(1.0)
                else:
                    sampled_service_time = random.expovariate(5.0)

            # Freeze until that time has passed
            yield self.env.timeout(sampled_service_time)
            
            # Calculate time customer was queueing
            out_of_system = self.env.now
            sojourn_time = out_of_system - start_queue
            customer.queue_time = end_queue - start_queue
            df_to_add = pd.DataFrame({"Customer ID":[customer.id], "Start Queueing": [start_queue],
                                     "End Queueing": [end_queue], "Queueing Time": [customer.queue_time],
                                     "Sojourn Time": [sojourn_time]})
            df_to_add.set_index("Customer ID", inplace=True)
            self.results_df = self.results_df.append(df_to_add)
            
    def calculate_mean_queue_time(self):
        self.mean_queue_time = self.results_df["Queueing Time"].mean()
        
    def write_run_results(self):
        with open("trial_results.csv", "a") as f:
            writer = csv.writer(f, delimiter=",")
            results_to_write = [self.run_number, self.mean_queue_time]
            writer.writerow(results_to_write)
            
    def run(self):
        self.env.process(self.generate_arrivals())
        self.env.run(until=g.sim_duration)
        # Calculate run results
        self.calculate_mean_queue_time()
        self.write_run_results()
      
            
   

In [6]:
for run in range(g.number_of_runs):
    mu_queue_model = SJF_queue(run)
    mu_queue_model.run()

In [7]:
mu_queue_model.results_df

,Start Queueing,End Queueing,Queueing Time,Sojourn Time
Customer ID,,,,
1,0.000000,0.000000,0.000000,3.507676
3,2.571673,3.507676,0.936003,1.063312
2,0.695001,3.634985,2.939984,4.462985
6,4.391795,5.157986,0.766191,0.872970
4,3.420918,5.264765,1.843847,3.033387
...,...,...,...,...
209,191.630140,192.704869,1.074729,2.017838
211,191.909588,193.647977,1.738389,2.675369
207,191.240714,194.584957,3.344243,4.950720


## Different service rate distribution on M/D/n and M/D/1

## Long-tail distribution